In [2]:
from delierium.Infinitesimals import prolongationODE, infinitesimalsODE, Janet_Basis_from_ODE
from delierium.JanetBasis import Janet_Basis
from delierium.helpers import is_derivative, order_of_derivative, eq
from delierium.MatrixOrder import Context
import functools
from operator import mul
from pprint import pprint
from delierium.helpers import latexer
from sage.misc.html import html
from IPython.core.debugger import set_trace
from IPython.display import Math

In [7]:
#https://ask.sagemath.org/question/7929/computing-variational-derivatives/
from sage.all import *
import sage.symbolic.operators

def is_op_du(expr_op, u):
    is_derivative = isinstance(
        expr_op,
        sage.symbolic.operators.FDerivativeOperator
    )

    if is_derivative:
        # Returns True if the differentiated function is `u`.
        return expr_op.function() == u.operator()

    else:
        return False


def iter_du_orders(expr, u):
    for sub_expr in expr.operands():
        if sub_expr == []:
            # hit end of tree
            continue

        elif is_op_du(sub_expr.operator(), u):
            # yield order of differentiation
            yield len(sub_expr.operator().parameter_set())

        else:
            # iterate into sub expression
            for order in iter_du_orders(sub_expr, u):
                yield order


def func_diff(L, u_in):
    # `u` must be a callable symbolic expression
    # in one variable.
    if len(u_in.variables()) == 1:
        x = u_in.variables()[0]
        u = u_in.function(x)
    else:
        raise TypeError

    # This variable name must not collide
    # with an existing one.
    # I use an empty string in hopes that
    # nobody else does this...
    t = SR.symbol()

    result = SR(0)

    # `orders` is the set of all
    # orders of differentiation of `u`
    orders = set(iter_du_orders(L, u)).union((0,))

    for c in orders:
        du = u(x).diff(x, c)
        sign = Integer(-1)**c

        # Temporarily replace all `c`th derivatives of `u` with `t`;
        # differentiate; then substitute back.
        dL_du = L.subs({du:t}).diff(t).subs({t:du})

        # Append intermediate term to `result`
        result += sign * dL_du.diff(x, c)

    return result

u = function('u')(x)
L = u^3 + (1/2)*u.diff(x)^2 + u^2 * sqrt(u + u*x)
rr=func_diff(L, u)
rr.show()
@functools.cache
def eta_n(n, f1, f2):
    if n == 1:
        return func_diff(f1, x) - func_diff(y, x) * func_diff(f2, x)
    else:
        return func_diff(eta_n(n-1, f1, f2), x) - diff(y, x, n) * func_diff(f2, x)

x = var("x")
y = function("y")(x)
eta=function("eta", latex_name=r"\eta")(y, x)
xi=function("xi", latex_name=r"\xi")(y, x)

eta_n(1, eta, xi)

1/2*(x + 1)*u(x)^2/sqrt(x*u(x) + u(x)) + 3*u(x)^2 + 2*sqrt(x*u(x) + u(x))*u(x) - diff(u(x), x, x)

-(D[0](xi)(y(x), x)*diff(y(x), x) + D[1](xi)(y(x), x))*diff(y(x), x) + D[0](eta)(y(x), x)*diff(y(x), x) + D[1](eta)(y(x), x)

In [4]:
r2=[eta_n(k, eta, xi) for k in range(1,11)]

In [5]:
for _ in r2:
    print(len(_.expand().operands()))

4
9
17
29
47
73
110
162
234
332


In [21]:
r2[0].show()

-(D[0](xi)(y(x), x)*diff(y(x), x) + D[1](xi)(y(x), x))*diff(y(x), x) + D[0](eta)(y(x), x)*diff(y(x), x) + D[1](eta)(y(x), x)